## Google Collab Dependencies

In [ ]:
!curl -O https://raw.githubusercontent.com/dnoliver/fraud-detection-sample/master/utils.py
!curl -O https://raw.githubusercontent.com/dnoliver/fraud-detection-sample/master/dataset_generation.py
!curl -O https://raw.githubusercontent.com/dnoliver/fraud-detection-sample/master/feature_transformation.py
!curl -O https://raw.githubusercontent.com/dnoliver/fraud-detection-sample/master/logistic_regression.py
!curl -O https://raw.githubusercontent.com/dnoliver/fraud-detection-sample/master/convolutional_neural_network.py
!pip install skorch

## Data Set Generation

In [ ]:
from dataset_generation import main as dataset_generation

dataset = dataset_generation()

In [ ]:
dataset

## Feature Transformation

In [ ]:
from feature_transformation import main as feature_transformation

dataset = feature_transformation()

In [ ]:
dataset

## Logistic Regression

In [ ]:
from logistic_regression import main as logistic_regression

performance, execution_time, summary = logistic_regression()

In [ ]:
performance

In [ ]:
print(f"Execution Time {execution_time[0]}")

In [ ]:
summary

## Convolutional Neural Network

In [ ]:
from convolutional_neural_network import main as convolutional_neural_network

performance, execution_time, summary = convolutional_neural_network()

In [ ]:
performance

In [ ]:
print(f"Execution Time {execution_time[0]}")

In [ ]:
summary